In [1]:
import json
from elasticsearch import Elasticsearch
import urllib3
import os


elstic_password = os.getenv('ELASTIC_PASSWORD')

es = Elasticsearch(
    ['https://localhost:9200'],
    basic_auth=('elastic', elstic_password),  #"T0DK*rhRoeIBBmxiD=vX"
    verify_certs=False
)

urllib3.disable_warnings()

C:\Users\linus\anaconda3\Lib\site-packages\elasticsearch\_sync\client\__init__.py:399: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


### Indexing Documents into Elasticsearch for BM25 Search

In [28]:
from pathlib import Path
import os

# Define the path for the source directory
source_directory = Path('C:/Users/linus/big_data/pubmed/test_chunk/')

# Iterate through each file in the source directory
for file_name in os.listdir(source_directory):
    if file_name.endswith('.jsonl'):
        source_file = source_directory / file_name
        
        # Open and read the JSONL file containing the PubMed documents
        with open(source_file, 'r') as json_file:
            for line in json_file:
                doc = json.loads(line)
                # Index the document into Elasticsearch
                res = es.index(index="pubmed_index", document=doc)
                # Uncomment the next line if you want to see the indexing result for each document
                # print(res['result'])
    
print('Indexing complete')

Indexing complete


In [54]:
count_result = es.count(index='pubmed_emb_index')

# Print the count
print(count_result['count'])

133379


In [8]:
# Define a search query
def bm25_search(query: str, k: int = 5):
    query = {
        "size": k,
        "query": {
            "match": {
                "content": f"{query}"
            }
        }
    }
    # Elasticsearch nutzt standardmässig das BM25-Modell, um die Relevanz der Dokumente zu berechnen
    return es.search(index='pubmed_index', body=query)
    
# Perform a search
results = bm25_search("List signaling molecules (ligands) that interact with the receptor EGFR?", k=10)

# Print the results
for hit in results['hits']['hits']:
    print(f"Score: {hit['_score']}, Title: {hit['_source']['content']}")

Score: 25.100676, Title: Colchicine, a plant alkaloid, is a potent inhibitor of mitosis and other physiological processes that involve microtubules. These effects are mediated by the specific binding of colchicine to a high-affinity receptor site on tubulin, the major protein of microtubules. It seemed possible that the colchicine site on tubulin might also be the receptor for endogenous cellular molecules. We now report that mammalian brain does, in fact, contain a class of molecules that interact with the colchicine site on tubulin. Tubulin-agarose affinity chromatography has been used to isolate factors from soluble extracts of bovine brain that interact with tubulin. Certain of these factors inhibit the binding of [3H]colchicine to tubulin. Using the inhibition of colchicine binding as an assay, I have partially resolved at least two species. One is a protein and another may be a heat-stable peptide. In parallel to the action of colchicine, the smaller species inhibits the assembly

### Indexing Documents with Embeddings into Elasticsearch for Vector Similarity Search

initialize the index with the appropriate mapping for the dense vector field.

In [31]:
# Define the index name
index_name = "pubmed_emb_index"

# Check if the index already exists
if not es.indices.exists(index=index_name):
    # Define the mapping
    mapping = {
        "mappings": {
            "properties": {
                "embeddings": {"type": "dense_vector", "dims": 768}  # Adjust the dimension size as needed
                # Add other field mappings as necessary
            }
        }
    }
    
    # Create the index with the defined mapping
    es.indices.create(index=index_name, body=mapping)

load the JSONL files containing the PubMed documents, extract the embeddings, and index the documents into Elasticsearch.

In [ ]:
from pathlib import Path
import os
import json
from tqdm import tqdm

# Define the path for the source directory
source_directory = Path('C:/Users/linus/big_data/pubmed/chunk_embedded/')

# Check if the source directory exists
if not source_directory.exists():
    print("The source directory does not exist.")
else:
    # Iterate through each file in the source directory
    for file_name in tqdm(os.listdir(source_directory)):
        if file_name.endswith('.jsonl'):
            source_file = source_directory / file_name
            
            # Open and read the JSONL file containing the PubMed documents
            with open(source_file, 'r') as json_file:
                for line in json_file:
                    try:
                        doc = json.loads(line)
                        # Ensure the document includes an 'embeddings' field with the vector data
                        # Index the document into Elasticsearch
                        res = es.index(index=index_name, document=doc)
                        # Uncomment the next line to see the indexing result for each document
                        # print(res['result'])
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON: {e}")
                    except Exception as e:
                        print(f"An error occurred: {e}")

    print('Indexing complete')

 50%|█████     | 5/10 [1:08:00<1:09:29, 833.83s/it]

An error occurred: Connection timed out


perform a vector similarity search using the cosine similarity between the query vector and the embeddings in the indexed documents.

In [5]:
from Embedding import TextEmbedder
embedder = TextEmbedder()

In [10]:
# Define the index name
index_name = "pubmed_emb_index"


def query_to_vector(text, embedder):
    embedding = embedder.embed(text)
    return embedding


def cosine_similarity(index, query: str, k: int = 5):
    query_vector = query_to_vector(query, embedder)
    
    query = {
        "size": k,  # Anzahl der zurückzugebenden Ergebnisse
        "query": {
            "script_score": {
                "query": {"match_all": {}},  # Sie können dies durch eine spezifischere Abfrage ersetzen
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'embeddings') + 1.0",
                    # +1.0, um sicherzustellen, dass alle Werte positiv sind
                    "params": {"query_vector": query_vector}
                }
            }
        }
    }
    # Elasticsearch nutzt standardmässig das BM25-Modell, um die Relevanz der Dokumente zu berechnen
    return es.search(index=index, body=query)


# Führen Sie die Abfrage aus
results = cosine_similarity(index_name, "List signaling molecules (ligands) that interact with the receptor EGFR?", k=10)

for hit in results['hits']['hits']:
    print(f"Score: {hit['_score']},  PMID: {hit['_source']['PMID']}, Abstract: {hit['_source']['title']}")

Score: 1.9132408,  PMID: 233264, Abstract: Hormonal regulation of peptide receptors and target cell responses.
Score: 1.9074038,  PMID: 219362, Abstract: Mouse lymphoma cells with mutations of cyclic AMP-dependent protein kinase.
Score: 1.9039831,  PMID: 200850, Abstract: [Molecular biology of tumor viruses].
Score: 1.9023585,  PMID: 179753, Abstract: Diseases associated with herpesviruses.
Score: 1.9007919,  PMID: 208416, Abstract: Receptor dysfunction and hormone resistance in human diseases--a review.
Score: 1.9004114,  PMID: 187763, Abstract: Elevated adenosine 3',5'-cyclic monophosphate levels in human and animal tumors in vivo.
Score: 1.8994532,  PMID: 91568, Abstract: [Clinical valence of alpha-fetoprotein (author's transl)].
Score: 1.899434,  PMID: 204384, Abstract: [Hormonal factors in the prediction of hormonedependency of breast cancer].
Score: 1.899276,  PMID: 220539, Abstract: Local aggregation of hormone-receptor complexes is required for activation by epidermal growth fa

In [15]:
response = es.indices.stats(index='pubmed_emb_index')

index_size = response['_all']['total']['store']['size_in_bytes']

print(f"Die Größe des Indexes ist {index_size/1000000000} Bytes.")

Die Größe des Indexes ist 2.239613243 Bytes.
